In [1]:
#mecabで分かち書きしながら、tokenizeする


# mecabによる分かち書き

In [2]:
import json
#コーパスの読み込み
target_path="/data/hatakeyama/python/llm_corpus/corpus_scale_200.jsonl"
lines=[]
with open(target_path, "r") as f:
    for line in f:
        lines.append(json.loads(line)["text"])

#一時ファイルに書き込み
with open("../data/corpus_text.txt","w") as f:
    f.write("")
with open("../data/corpus_text.txt","a") as f:
    for line in lines:
        f.write(line+"\n")

In [3]:
#わかちがき
!mecab -F"%M||||" -E"\n" -b 100000 < ../data/corpus_text.txt  > ../data/corpus_text.txt.tok

input-buffer overflow. The line is split. use -b #SIZE option.
input-buffer overflow. The line is split. use -b #SIZE option.
input-buffer overflow. The line is split. use -b #SIZE option.
input-buffer overflow. The line is split. use -b #SIZE option.
input-buffer overflow. The line is split. use -b #SIZE option.
input-buffer overflow. The line is split. use -b #SIZE option.
input-buffer overflow. The line is split. use -b #SIZE option.
input-buffer overflow. The line is split. use -b #SIZE option.
input-buffer overflow. The line is split. use -b #SIZE option.
input-buffer overflow. The line is split. use -b #SIZE option.
input-buffer overflow. The line is split. use -b #SIZE option.
input-buffer overflow. The line is split. use -b #SIZE option.
input-buffer overflow. The line is split. use -b #SIZE option.
input-buffer overflow. The line is split. use -b #SIZE option.
input-buffer overflow. The line is split. use -b #SIZE option.
input-buffer overflow. The line is split. use -b #SIZE 

In [4]:
len(lines)

1216549

# 学習

In [1]:
from types import SimpleNamespace
import sentencepiece as spm
import sys
import yaml
import os
from special_token_list import *


def yaml_to_namespace(yaml_path):
    with open(yaml_path, 'r') as file:
        # YAMLファイルを辞書として読み込む
        data = yaml.safe_load(file)
        # 辞書をSimpleNamespaceに変換
        return recursive_namespace(data)


def recursive_namespace(data):
    if isinstance(data, dict):
        # 再帰的に辞書の各要素をSimpleNamespaceに変換
        return SimpleNamespace(**{k: recursive_namespace(v) for k, v in data.items()})
    elif isinstance(data, list):
        # リストの要素も変換
        return [recursive_namespace(v) for v in data]
    else:
        # その他のデータ型はそのまま返す
        return data


args = yaml_to_namespace('../30tokenize/config.yaml')

args.input="../data/corpus_text.txt.tok"
#args.vocab_size=3000
1

1

In [2]:
print("begin tokenizing...")

spm.SentencePieceTrainer.train(
        input=args.input,
        model_prefix=args.model_prefix,
        vocab_size=args.vocab_size,
        character_coverage=args.character_coverage,
        model_type=args.model_type,
        num_threads=args.num_threads,
        train_extremely_large_corpus=args.train_extremely_large_corpus,
        user_defined_symbols=[
            BOS_TOKEN,
            EOS_TOKEN,
            PAD_TOKEN,
            CLS_TOKEN,
            SEP_TOKEN,
            EOD_TOKEN,
            MASK_TOKEN,
            NEWLINE_TOKEN,
            EXTRA_TOKEN1,
            EXTRA_TOKEN2,
            EXTRA_TOKEN3,
            EXTRA_TOKEN4,
            EXTRA_TOKEN5,
            EXTRA_TOKEN6,
            EXTRA_TOKEN7,
            EXTRA_TOKEN8,
        ],  # Note: `NEWLINE_TOKEN` is needed in `user_defined_symbols`.
        byte_fallback=True,
        split_digits=True,
        allow_whitespace_only_pieces=True,
        remove_extra_whitespaces=False,
        pretokenization_delimiter="||||",
    )


begin tokenizing...


In [3]:

import sentencepiece as spm
#test
model_path="../data/tokenizers/tokenizer_scale200.model"
sp = spm.SentencePieceProcessor(model_file=model_path)


In [6]:

text=" 価格詳細, 日本の正式代理店で購入する必要がある"
text="he is a good man"
text="import pandas as pd  from tqdm import tqdm"
text="""
def yaml_to_namespace(yaml_path):
    with open(yaml_path, 'r') as file:
        # YAMLファイルを辞書として読み込む
        data = yaml.safe_load(file)
        # 辞書をSimpleNamespaceに変換
        return recursive_namespace(data)
"""

text="これはテストです。"
text="吾輩は猫である｡名前はまだない"
text="吾輩は猫である｡\n名前はまだない"
(sp.encode(text, out_type=str)),(sp.encode(text))

(['▁', '吾', '輩', 'は', '猫', 'である', '。', '\n', '名前', 'はまだ', 'ない'],
 [272, 6324, 1713, 276, 1642, 375, 270, 8, 1132, 2315, 326])

In [11]:
t="ああ\nああ"
tokens=sp.encode(t)
t,tokens,sp.decode(tokens)

('ああ\nああ', [8722, 8, 2721], 'ああ\nああ')